In [50]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from subprocess import check_output

# Pipeline at a glance:
load data -> cleaning/preprocessing -> feature eng -> format data for modelling -> fit model -> evaluate model -> generate submission file

## Step 1: Load data

In [51]:
df_train = pd.read_csv('../input/train.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [52]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [53]:
df_test = pd.read_csv('../input/test.csv')
df_test_duplicate = df_test
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [54]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

## Step 2: Basic cleaning and preprocessing

In [55]:
# label encode sex
le = LabelEncoder()
le.fit(df_train.Sex)
df_train.loc[:,'Sex'] = le.transform(df_train.Sex)
df_test.loc[:,'Sex'] = le.transform(df_test.Sex)

In [56]:
# Fill in the missing values
mAge = pd.concat((df_train.Age, df_test.Age), axis=0).mean()
df_train.loc[:, 'Age'] = df_train.Age.fillna(mAge)
df_test.loc[:, 'Age'] = df_test.Age.fillna(mAge)

## Step 3: Feature engineering

In [57]:
df_train = df_train.replace(np.nan, 'S', regex=True)
df_test = df_test.replace(np.nan, 'S', regex=True)

In [58]:
df_train['age_binned'] = pd.cut(df_train['Age'], [0, 16, 32, 48, 64, 200], labels=[0,1,2,3,4], retbins=False)
df_test['age_binned'] = pd.cut(df_test['Age'], [0, 16, 32, 48, 64, 200], labels=[0,1,2,3,4], retbins=False)

df_train['family_size'] = df_train['SibSp'] + df_train['Parch']
df_test['family_size'] = df_test['SibSp'] + df_test['Parch']

df_train['is_alone'] = np.where(df_train['family_size']==0, 1, 0)
df_test['is_alone'] = np.where(df_test['family_size']==0, 1, 0)

le.fit(df_train.Embarked)
df_train.loc[:,'Embarked'] = le.transform(df_train.Embarked)
df_test.loc[:,'Embarked'] = le.transform(df_test.Embarked)

df_train = df_train.drop(['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin'], axis=1)
df_test = df_test.drop(['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin'], axis=1)

df_train

,Survived,Pclass,Sex,Embarked,age_binned,family_size,is_alone
0,0,3,1,2,1,1,0
1,1,1,0,0,2,1,0
2,1,3,0,2,1,0,1
3,1,1,0,2,2,1,0
4,0,3,1,2,2,0,1
5,0,3,1,1,1,0,1
6,0,1,1,2,3,0,1
7,0,3,1,2,0,4,0
8,1,3,0,2,1,2,0
9,1,2,0,0,0,1,0


## Step 4: Format data for modelling

In [59]:
# Split data into X and y, and select features to use
X = df_train.loc[:,['age_binned', 'Sex', 'Pclass', 'Embarked', 'family_size', 'is_alone']]
y = df_train.loc[:,'Survived']

In [60]:
# Split data X and y into train and val sets
X_test = df_test.loc[:,['age_binned', 'Sex', 'Pclass', 'Embarked', 'family_size', 'is_alone']]
X_train, X_valid, y_train, y_valid = train_test_split(
      X, y, test_size=0.2, random_state=42)

## Step 5: Train (or fit) the model

In [61]:
# Fit logistic regression
LR = LogisticRegression()
LR.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Step 6: Evaluate the model

In [62]:
# define a scoring function
def acc(y: np.array, y_pred: np.array) -> float:
    return np.sum(y_pred==y)/len(y)

In [63]:
# Use model to predict validation set
y_pred_valid = LR.predict(X_valid)

In [64]:
print('Validation set accuracy', acc(y_valid, y_pred_valid), '%')

Validation set accuracy 0.8044692737430168 %


## Step 7: Generate Submission

In [65]:
# Predict for test set

y_pred_test = LR.predict(X_test)

# Create a Kaggle submission
sub = pd.DataFrame({'PassengerId': df_test_duplicate['PassengerId'],
                    'Survived': y_pred_test})

sub.to_csv('kaelan.csv', index=False)